In [11]:
import mmcv
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import os
import numpy as np

In [3]:

v = mmcv.VideoReader("test_video.mp4")
v.cvt2frames("frame_dir/in")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 150/150, 29.8 task/s, elapsed: 5s, ETA:     0s


In [4]:
model = init_detector(config="../work_dir/balloon.py", checkpoint="../work_dir/latest.pth")

load checkpoint from local path: ../work_dir/latest.pth


In [14]:
img_path = "../data/balloon/val/3825919971_93fb1ec581_b.jpg"
result = inference_detector(model, img_path)
show_result_pyplot(model, img_path, result, score_thr=0.85, out_file="img.jpg")

In [7]:
def transform_img(in_path, out_path, score_thr=0.85):
    for file_name in os.listdir(in_path):
        img_path = os.path.join(in_path, file_name)
        result = inference_detector(model, img_path)
        bbox, segm = result
        bbox = np.array(bbox[0])
        segm = np.array(segm[0])
balloon_cnt = bbox[bbox[:, -1] > score_thr].shape[0]
data = mmcv.imread(img_path)  # -> ndarray
tmp = data.copy()
# Gray = (Red * 0.3 + Green * 0.59 + Blue * 0.11)
# 'bgr'
data[:, :, 0] = data[:, :, 2] * 0.3 + data[:, :, 1] * 0.59 + data[:, :, 0] * 0.11
data[:, :, 1] = data[:, :, 0]
data[:, :, 2] = data[:, :, 0]
for balloon in segm[:balloon_cnt]:
    data[balloon] = tmp[balloon]
mmcv.imwrite(data, os.path.join(out_path, file_name))


transform_img("frame_dir/in", "frame_dir/out")

In [8]:
mmcv.frames2video(frame_dir="frame_dir/out", video_file="video_out.mp4")
print("--")

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 125/125, 23.2 task/s, elapsed: 5s, ETA:     0s
--
